In [ ]:
import json

with open("cleaned.json") as f:
  courses = json.load(f)

In [ ]:
for course in courses:
  try:
    with open("data/"+course['course_dept'] + course['course_number'] + course['course_title']+".txt", "w") as file:
        stringToWrite = ""
        for key, value in course.items():
            if value not in [None, ""]:
                stringToWrite += f"{key}: {value}\n"
        file.write(stringToWrite)
        print(stringToWrite)
  except:
    continue

In [ ]:

import PIL
!pip uninstall Pillow
!pip install --upgrade Pillow
print(PIL.__version__)



In [ ]:
# Requirement
!pip install openai -q
!pip install langchain -q
!pip install chromadb -q
!pip install tiktoken -q
!pip install pypdf -q
!pip install unstructured[local-inference] -q
!pip install gradio -q

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

import os
os.environ["OPENAI_API_KEY"] = ""
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0,model_name="gpt-4")

# Data Ingestion
from langchain.document_loaders import DirectoryLoader
excel_loader = DirectoryLoader('./data', glob="*.txt")
loaders = [excel_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

# Chunk and Embeddings
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)



In [ ]:
# Initialise Langchain - Conversation Retrieval Chain
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), vectorstore.as_retriever())



In [ ]:
# Front end web app
import gradio as gr
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    chat_history = []
    
    def user(user_message, history):
        print("CHAT HISTORY ", chat_history)
        print("HISTORY", history)
        history = [tuple(lst) for lst in history]
        # Get response from QA chain
        response = qa({"question": user_message, "chat_history": history})
        # Append user message and response to chat history
        history.append((user_message, response["answer"]))

        return gr.update(value=""), history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

CHAT HISTORY  []
HISTORY []
CHAT HISTORY  []
HISTORY [['What languages are taught at the university?', 'The university offers courses in several languages, including Arabic, French, Spanish, and others. The specific languages offered may vary each term, so it’s best to check the course catalog or schedule for the current offerings.']]
CHAT HISTORY  []
HISTORY [['What languages are taught at the university?', 'The university offers courses in several languages, including Arabic, French, Spanish, and others. The specific languages offered may vary each term, so it’s best to check the course catalog or schedule for the current offerings.'], ['What programming languages are taught?', 'The university offers Spanish, Arabic, French, and Chinese. You can find more information about these courses, including course descriptions, credit hours, and prerequisites, in the university’s course catalog or on the university’s website.']]
CHAT HISTORY  []
HISTORY [['What languages are taught at the univ

In [ ]:
""